# Import modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import datetime

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# ex: from src.create_model_pickle import *


# Import data

In [ ]:
# Use our json to dataframe conversion class
from src.pipeline import pipeline_json
pj = pipeline_json('../data/data.json')

In [ ]:
df = pj.convert_to_df()
df.info()

'''
0s or 1s :
'show_map', 'fb_published', 'has_logo', , 'has_analytics'


integers or floats:
'gts', 'sale_duration', 'org_twitter', 'org_facebook'

'y' or 'n' :
'listed'
yn_map = {'y': 1, 'n' : 0}
'''




bin_dict = {0 : False, 1 : True}
bin_cols = ['show_map', 'fb_published', 'has_logo', 'has_analytics']
for col in bin_cols:
    df[col] = df[col].map(bin_dict)

yn_dict = {'y': 1, 'n' : 0}
yn_cols = ['listed']
for col in yn_cols:
    df[col] = df[col].map(yn_dict)


bin_nan_cols = ['has_header']
for col in bin_nan_cols:
    df[col] = df[col].map(lambda x: x if not np.isnan(x) else -1)
    df[col].astype('int', copy=True)

payout_dict = {'CHECK': 1, 'ACH' : 1, '':0}
df['payout_type'] = df['payout_type'].map(payout_dict)


In [ ]:
df.info()

In [ ]:
df['has_header'][:10]
    